In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, getpass
#lari_id='fdb573a66f50' ## lari_ccblin008
lari_id='19b775bb238f' ## lari_g5ub
#lari_id='cb48a51bf9e5' ## lari_dub
#lari_id=''
if lari_id:
    print('Enter processing passcode for lari node {}'.format(lari_id))
    lari_passcode=getpass.getpass()
else:
    lari_passcode=''
os.environ['LARI_ID']=lari_id
os.environ['LARI_PASSCODE']=lari_passcode

Enter processing passcode for lari node 19b775bb238f


In [3]:
import os
import sys
dir0 = os.path.split(os.getcwd())[0]
if dir0 not in sys.path:
    sys.path.append(dir0)

In [4]:
from mountainsort4_v1 import sort_dataset as ms4_sort_dataset
from summarize_sorting_results import summarize_sorting_results

In [5]:
output_base_dir=os.getcwd()+'/output'
#import shutil
#if os.path.exists(output_base_dir):
#    shutil.rmtree(output_base_dir)
if not os.path.exists(output_base_dir):
    os.mkdir(output_base_dir)

In [6]:
datasets=[]
datasets.append({
    "id":"synth_tetrode_30min",
    "raw_path":"kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_tetrode_30min"
});
datasets.append({
    "id":"synth_tetrode_120min",
    "raw_path":"kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_tetrode_120min"
});
#datasets.append({
#    "id":"synth_16ch_30min",
#    "raw_path":"kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_16ch_30min"
#});

#datasets=[datasets[0]]

In [7]:
def ms4_thr3(dataset_dir,output_dir):
    ms4_sort_dataset(
        dataset_dir=dataset_dir,
        output_dir=output_dir,
        adjacency_radius=-1,
        detect_threshold=3,
        opts={'verbose':'minimal'}
    )
    
def ms4_thr4(dataset_dir,output_dir):
    ms4_sort_dataset(
        dataset_dir=dataset_dir,
        output_dir=output_dir,
        adjacency_radius=-1,
        detect_threshold=4,
        opts={'verbose':'minimal'}
    )

algs=[]
algs.append({
    "name":"ms4_thr3",
    "run":ms4_thr3
})
algs.append({
    "name":"ms4_thr4",
    "run":ms4_thr4
})

In [8]:
runs=[]
for i_ds in range(len(datasets)):
    for i_alg in range(len(algs)):
        runs.append({"dataset":datasets[i_ds],"alg":algs[i_alg]})
runs=[runs[0]]

In [9]:
def handle_run(run):
    DS=run['dataset']
    ALG=run['alg']
    print(':::: Applying '+ALG['name']+' to '+DS['id'])
    dsdir=DS['raw_path']
    dsid=DS['id']
    algname=ALG['name']
    output_dir=output_base_dir+'/'+dsid+'--'+algname
    ALG['run'](
        dataset_dir=dsdir,
        output_dir=output_dir
    )
    summarize_sorting_results(
        sorting_output_dir=output_dir,
        output_dir=output_dir+'/summary',
        opts={'verbose':'minimal'}
    )

In [11]:
import multiprocessing
pool = multiprocessing.Pool(len(runs))
pool.map(handle_run, runs) 

:::: Applying ms4_thr3 to synth_tetrode_30min
RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_tetrode_30min/raw.mda --parameters freq_max:6000 freq_min:300 samplerate:30000 --outputs timeseries_out:/home/magland/src/mountainsort_examples/spikeforest/output/synth_tetrode_30min--ms4_thr3/filt.mda.prv --verbose=minimal


KeyboardInterrupt: 

Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-9-5bb80d2f91a7>", line 11, in handle_run
    output_dir=output_dir
  File "<ipython-input-7-4e47f617d9c1>", line 7, in ms4_thr3
    opts={'verbose':'minimal'}
  File "/home/magland/src/mountainsort_examples/mountainsort4_v1.py", line 20, in sort_dataset
    opts=opts
  File "/home/magland/src/mountainsort_examples/mountainsort4_v1.py", line 83, in bandpass_filter
    opts
  File "/home/magland/.local/lib/python3.5/site-packages/mountainlab_pytools/mlproc/mlproc_impl.py", line 239, in